In [ ]:
#!jupyter nbconvert --to=python MZ_DIN_WOA_detailed_comparison_python3.ipynb

# After changes in the script using Jupyter notebook, save the notebook, run this line, and clear the output.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from netCDF4 import Dataset
from scipy.interpolate import griddata
import skill_metrics as sm
import cartopy.crs as ccrs
import pickle

import pyfesom2 as pf


#import sys
#sys.path.append('home/ollie/mozeisin/')
#from PiecewiseNorm import PiecewiseNorm
#modulepath = '/home/ollie/mozeisin/py_recom/GlobalAssessment'
#sys.path.append('/home/ollie/mozeisin/pyfesom2/pyfesom2/')


In [ ]:
# cartopy specification -------------------------------------------------------------------------------------
#p_PC = ccrs.PlateCarree()
#p_Rb = ccrs.Robinson()

In [ ]:
# run specification -------------------------------------------------------------------------------------
runname = 'mo1'
runid = runname
var = 'Dia'

resultpath = '/work/ollie/mozeisin/results/f2r1.2/' + runname
savepath = '/home/ollie/mozeisin/evaluation/mo_files'
htmlname     =  runname+'_DIN_WOA.html'
htmlpath = '/home/ollie/mozeisin/evaluation/mo_files/html/'
export_html = False

meshpath = '/work/ollie/mozeisin/mesh/mesh_fesom2.0/core2_meanz'

# period of analysis ------------------------------------------------------------------------------------

first_year = 1975
last_year  = 1984

years = np.arange(first_year,last_year+1,1)

In [ ]:
# load FESOM mesh -------------------------------------------------------------------------------------
mesh       = pf.load_mesh(meshpath)
#print(mesh.zlev)

## World Ocean Atlas

In [ ]:
# load WOA DIN -------------------------------------------------------------------------------------
ncfileDIN                = '/work/ollie/projects/MarESys/evaluation/woa13_all_n00_01.nc'
#!ncdump -h $ncfileDIN

labelwoa = 'World Ocean Atlas DIN'
unitwoa = 'DIN [$\mu$mol L$^{-1}$]'

f          = Dataset(ncfileDIN, 'r')
DepthRaw   = -f.variables['depth'][:]                                
lonwoa     =  f.variables['lon'][:]
latwoa     =  f.variables['lat'][:]
Timewoa     =  f.variables['time'][:]
DINwoa     =  f.variables['n_an'][:]
DINwoa     = np.squeeze(DINwoa)
DINwoa     = np.ma.filled(DINwoa, np.nan)                            # From masked array to numpy array

X360, Y180 = np.meshgrid(lonwoa, latwoa)

print('WOA Shape DINwoa: {0} \nDepths n = {1} \nValues: {2}'.format(np.shape(DINwoa),len(DepthRaw),DepthRaw.view())) # depth, lat, lon

# compare to FESOM mesh
print('\nFESOM: \nDepths n = {0} \nValues: {1}'.format(len(mesh.zlev), mesh.zlev))

In [ ]:
# first overview -----------------------------------------------------------------------------------
fig = plt.figure(figsize= (7,7))
ax = plt.subplot()#projection = p_Rb)
im = ax.pcolor(X360, Y180, DINwoa[0,:,:])#, transform = p_PC)
cbar = fig.colorbar(im, orientation = 'horizontal')
cbar.set_label(unitwoa) 
plt.title(labelwoa)


In [ ]:
# interpolation specification ------------------------------------------------------------------------------------
use_pickle = False
pickle_file = 'TEMP_woa_din_interpolated'

if(use_pickle == False):
    # original py_recom code:
    #do2_int = np.zeros(len(mesh.z3))                                  # initializing target field for interpolation 
    #for k in range(0,len(mesh.zlevs)):

    # check maximum depth in WOA compared to FESOM
    dmin_woa = np.min(DepthRaw)
    dmin_fesom = np.min(mesh.zlev)

    if(dmin_woa <= dmin_fesom):
        print('***\nDepth greater in WOA ({0}) than in FESOM ({1})'.format(dmin_woa, dmin_fesom))
        ilev = len(mesh.zlev)
        max_zlev = mesh.zlev[ilev-1]
    else:
        print('***\nDepth greater in FESOM ({1}) than in WOA ({0})'.format(dmin_woa, dmin_fesom))
        ilev = np.where(mesh.zlev >= dmin_woa)
        ilev = ilev[0][-1]
        max_zlev = mesh.zlev[ilev]

    print('Please consider choosing max depth level {0} with max depth at {1}!\n***'.format(ilev,max_zlev))
    
    # storage container
    din_int = np.zeros((len(mesh.zlev)-1,len(mesh.x2)))
    #print(np.shape(din_int))
    
    for k in range(0,len(mesh.zlev)-1): # -1 because midlevel is used...
        lev = mesh.zlev[k] # current FESOM depth
        try:
            nextlev = mesh.zlev[k+1] # next level needed because FESOM data is defined for midlevel??
        except:
            print('reached bottom... no midlevel could be calculated')
        finally:
            midlev = (lev + nextlev )/2 # FESOM data is expected to be at midlevel?!
            #print(lev)
        ind1 = np.where(DepthRaw >= midlev)
        ind1 = ind1[0][-1]
        ind2 = np.where(DepthRaw < midlev)[0]

        if ind2.size > 0:                            # If we have not yet reached the bottom
            ind2 = ind2[0]                           # The index of the depth level below the current fesom level
            c    = DepthRaw[ind1]-DepthRaw[ind2]     # Difference in depth between the data value above and below the fesom depth
            c1   = DepthRaw[ind1]-midlev                # Difference between fesom depth and data depth above
            c2   = -(DepthRaw[ind2]-midlev)             # Difference between fesom depth and data depth below
            c1   = (c-c1)/c                          # Scaling coefficient for the depth above
            c2   = (c-c2)/c                          # Scaling coefficient for the depth below
        else:                                        # We have reached the bottom
            c1   = 1.
            c2   = 0.
            ind2 = ind1

        indZ  = np.where(mesh.zlev == lev)                               
        # original code:
        # indZ  = np.where(-mesh.z3 == lev)          # Find the mesh index of the current fesom depth
        indZ = indZ[0] 
        if(False):
            print('\nFESOM depth = {0} -> midlevel = {8}, WOA depths = {1}, {2} \nDepth indices: {3} {4},  FESOM index: {5} \nScaling c1 = {6}, c2 = {7}'.format(lev,DepthRaw[ind1],DepthRaw[ind2],ind1, ind2,indZ,c1,c2,midlev))


        aux1  = DINwoa[ind1,:,:]                     # Find the data above the current fesom depth
        aux2  = DINwoa[ind2,:,:]                     # Find the data below the current fesom depth
        aux   = np.squeeze(c1*aux1+c2*aux2)          # Scaling the data according to vertical distribution as found above
        ind   = np.squeeze(~np.isnan(aux)) 
        #print(np.shape(aux), np.shape(ind))

        # first interpolation to original grid to close empty parts
        aux = griddata((X360[ind], Y180[ind]), aux[ind], (X360, Y180), method='nearest')                             
        # 2D field without nans                           

        # second interpolation to FESOM grid
        din_int[indZ,:] = griddata((X360.ravel(), Y180.ravel()), aux.ravel(), 
                               (mesh.x2, mesh.y2), method='nearest')  
        # Final interpolated field

        if np.isnan(np.min(din_int)): print('WARNING: The interpolated field contains NaNs at depth',lev)                 # Testing if results contain NaNs. If yes, the routine needs adjustments

        #print('Depth: {0} min = {1} max = {2} mean = {3}'.format(lev,np.min(din_int), np.max(din_int), np.mean(din_int)))

    din_int = np.swapaxes(din_int,0,1) # adjust axes layout to FESOM output
    #print(np.shape(din_int))
    
    outfile = open(pickle_file, "wb")
    pickle.dump(din_int,outfile)    
    outfile.close()
    
else:
    try:
        ifile = open(pickle_file, "rb")
        din_int = pickle.load(ifile)
        ifile.close()
    except:
        print('pickle file does not exist!')

## FESOM output data

In [ ]:
# check variables
NCfesom = resultpath + '/DIN.mo1.1948.nc'
!ncdump -h $NCfesom

In [ ]:
labelfesom = 'FESOM DIN'
unitfesom = 'DIN [mmol m$^{-3}$]' # equals to mumol/L

# load data
DINfesom = pf.get_data(resultpath, "DIN", [1948,1949,1950], mesh, how="mean", compute=True, runid='mo1')
np.shape(DINfesom)

## Plot on FESOM mesh

In [ ]:
# get mesh index closest to desired depth
depth = -2000
i = pf.ind_for_depth(depth,mesh) 

# get midlevel depth
plot_depth = str((mesh.zlev[i]+mesh.zlev[i+1])/2)


In [ ]:
# plot interpolated FESOM data 
if(True):
    pf.plot(mesh, DINfesom[:,i], units=unitfesom + '\n at depth = {0}m'.format(plot_depth),
        mapproj='pc', # robinson projection takes more time!
        titles = labelfesom)
  

In [ ]:
# apply sea mask to WOA as in FESOM ----------------------------------------------------------------------------------
# assumption: there is no ocean where value in FESOM == 0
din_int_ma = np.copy(din_int)
din_int_ma[DINfesom == 0] = 0

if(True):
    # quick check of interpolated WOA data ----------------------------------------------------------------------------------
    pf.plot(mesh,din_int_ma[:,i],
        units=unitwoa + '\n at depth = {0}m'.format(plot_depth), #mapproj='rob',
        titles=labelwoa)    

## Comparison with mean over depth

In [ ]:
DINfesom_mean = np.mean(DINfesom, axis = 1)
din_int_ma_mean = np.mean(din_int_ma, axis = 1)
#print(np.shape(DINfesom_mean), np.shape(din_int_ma_mean))

if(True):
    print('\nPlotting DIN as mean over depth\nFESOM min = {0}, max = {1}\nWOA min = {2}, max = {3}'.format(
            np.nanmin(DINfesom_mean),np.nanmax(DINfesom_mean),
            np.nanmin(din_int_ma_mean),np.nanmax(din_int_ma_mean)))
    
    pf.plot(mesh, [din_int_ma_mean,DINfesom_mean, din_int_ma_mean-DINfesom_mean], 
            rowscol= (3,1),
            levels = (0,35,36),
            units=unitwoa, 
            mapproj='pc', # robinson projection takes more time!
            titles=['World Ocean Atlas: Interpolated DIN for initialization', 
                    'FESOM Dissolved Inorganic Nitrogen ({0}-{1})'.format(first_year,last_year),
                    'Initialization fields minus FESOM'],
            figsize = (20,20)
           )

## Statistics with mean over depth

In [ ]:
#plt_Taylor_norm?

In [ ]:
from plot_Taylor_normalized import plt_Taylor_norm

title = 'Taylor Diagram for DIN \n(mean over depth)'

plt_Taylor_norm(din_int_ma_mean,DINfesom_mean, title=title)


In [ ]:
# preparation of datasets
if np.isnan(np.min(din_int_ma_mean)): print('WARNING: The interpolated WOA field contains NaNs at depth')
if np.isnan(np.min(DINfesom_mean)): print('WARNING: The interpolated FESOM field contains NaNs at depth')


# get statistics only from ocean gridpoints (same mask assumption as above)
ind_stat = np.where(DINfesom_mean != 0)

taylor_stats1 = sm.taylor_statistics(din_int_ma_mean[ind_stat],DINfesom_mean[ind_stat])
sdev = np.array([taylor_stats1['sdev'][0]/taylor_stats1['sdev'][0], 
                 taylor_stats1['sdev'][1]/taylor_stats1['sdev'][0]])
crmsd = np.array([taylor_stats1['crmsd'][0], taylor_stats1['crmsd'][1]])
ccoef = np.array([taylor_stats1['ccoef'][0], taylor_stats1['ccoef'][1]])

label = ['Observation', 'Model']# at {0} m'.format(plot_depth)]

fig = plt.figure(figsize=(7,7), facecolor='w', edgecolor='k')
sm.taylor_diagram(sdev,crmsd,ccoef, styleOBS = '-', colOBS = 'r', markerobs = 'o',
                      titleOBS = 'observation', markerLabel = label,
                      markerLabelColor = 'c',
                      markerColor = 'c', markerLegend = 'on',
                      #tickRMS = range(0,5,1),
                      tickRMSangle = 135.0,
                      colRMS = 'm', styleRMS = ':', widthRMS = 2.0,
                      titleRMS = 'off', #tickSTD = range(0,1,2 ),
                      #axismax = 1.0, 
                      colSTD = 'b', styleSTD = '-.',
                      widthSTD = 1.0, titleSTD = 'on',
                      colCOR = 'k', styleCOR = '--', widthCOR = 1.0,
                      titleCOR = 'on')
plt.title('Taylor Diagram for DIN \n(mean over depth)', loc='right')    

## Comparison layerwise

In [ ]:
# plot WOA and FESOM ----------------------------------------------------------------------------------
depth_array = (0,50,200,1000,2000,4000)

for d in depth_array:
    # get mesh index closest to desired depth
    i = pf.ind_for_depth(d,mesh) 
    # get midlevel depth
    plot_depth = str((mesh.zlev[i]+mesh.zlev[i+1])/2)

    if(True):
        print('\nInput depth = {0}, plotting at depth = {1} m\nFESOM min = {2}, max = {3}\nWOA min = {4}, max = {5}'.format(
            d,plot_depth,
            np.nanmin(DINfesom[:,i]),np.nanmax(DINfesom[:,i]),
            np.nanmin(din_int_ma[:,i]),np.nanmax(din_int_ma[:,i])))
    
    pf.plot(mesh, [din_int_ma[:,i],DINfesom[:,i], din_int_ma[:,i]-DINfesom[:,i]], 
            rowscol= (1,3),
            levels = (0,45,40),
            units=unitwoa + '\n at depth = {0} m'.format(plot_depth), 
            mapproj='pc', # robinson projection takes more time!
            titles=[labelwoa, labelfesom, 'WOA - FESOM'],figsize = (20,20)
           )

## Statistics layerwise

In [ ]:
#sm.taylor_statistics?

In [ ]:
# preparation of datasets
if np.isnan(np.min(din_int)): print('WARNING: The interpolated WOA field contains NaNs at depth')
if np.isnan(np.min(DINfesom)): print('WARNING: The interpolated FESOM field contains NaNs at depth')


for d in depth_array:
    # get mesh index closest to desired depth
    i = pf.ind_for_depth(d,mesh) 
    # get midlevel depth
    plot_depth = str((mesh.zlev[i]+mesh.zlev[i+1])/2)

    # get statistics only from ocean gridpoints (same mask assumption as above)
    ind_stat = np.where(DINfesom[i,:] != 0)

    taylor_stats1 = sm.taylor_statistics(din_int[i,ind_stat],DINfesom[i,ind_stat])
    sdev = np.array([taylor_stats1['sdev'][0], taylor_stats1['sdev'][1]])
    crmsd = np.array([taylor_stats1['crmsd'][0], taylor_stats1['crmsd'][1]])
    ccoef = np.array([taylor_stats1['ccoef'][0], taylor_stats1['ccoef'][1]])
    
    label = ['Observation', 'Model']# at {0} m'.format(plot_depth)]

    fig = plt.figure(figsize=(7,7), facecolor='w', edgecolor='k')
    sm.taylor_diagram(sdev,crmsd,ccoef, styleOBS = '-', colOBS = 'r', markerobs = 'o',
                          titleOBS = 'observation', markerLabel = label,
                          markerLabelColor = 'c',
                          markerColor = 'c', markerLegend = 'on',
                          tickRMS = range(0,5,1), tickRMSangle = 135.0,
                          colRMS = 'm', styleRMS = ':', widthRMS = 2.0,
                          titleRMS = 'off', tickSTD = range(0,10,2 ),
                          axismax = 10.0, 
                          colSTD = 'b', styleSTD = '-.',
                          widthSTD = 1.0, titleSTD = 'on',
                          colCOR = 'k', styleCOR = '--', widthCOR = 1.0,
                          titleCOR = 'on')
    plt.title('Taylor Diagram for DIN at {0} m'.format(plot_depth), loc='right')

#### now export...

In [ ]:
if(export_html):
    %autosave 2
    import time
    time.sleep(15)
    !jupyter nbconvert MZ_DIN_WOA_detailed_comparison_python3.ipynb --output-dir=$htmlpath --output $htmlname --to html 